## Data Extraction

In [2]:
print('ok')

ok


In [3]:
%pwd

'c:\\Users\\SATHVIK\\Desktop\\Medical-Chatbot\\research'

In [ ]:
import os
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\SATHVIK\\Desktop\\Medical-Chatbot'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

c:\Users\SATHVIK\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
def pdf_loader(data):
    loader = DirectoryLoader(
        data,
        glob='*.pdf',
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

extracted_data = pdf_loader('data')
print(len(extracted_data))

637


## Data Filteration

In [13]:
from typing import List
from langchain.schema import Document

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [14]:
minimal_docs = filter_to_minimal_docs(extracted_data)

## Data Chunking

In [15]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [18]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")


Number of chunks: 5859


## Chunk Embedding